In [1]:
from text_utils import *

#current version
#export TF_BINARY_URL=https://storage.googleapis.com/tensorflow/linux/gpu/tensorflow_gpu-1.0.1-cp36-cp36m-linux_x86_64.whl
import tensorflow as tf
from tensorflow.contrib.rnn.python.ops import core_rnn_cell as rnn_cell
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn,dynamic_rnn
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import pickle as pkl
import pandas as pd
from os import walk
import h5py
import sys
sys.path.append('../tfmodels')
#from tf_object import *
from text_cnn import *
from collections import Counter

data_path = 'data'

In [2]:
f = open('{}/emotion_sents.dat'.format(data_path),'rb')
new_sentences = pkl.load(f)
label_str = pkl.load(f)
f.close()
df = pd.read_excel('{}/emotion.xlsx'.format(data_path))
maxlen = 140
phrase_dict = dict(zip(df.phrase, df.po1))
id_term_map = pkl.load(open('{}/term_map.dat'.format(data_path),'rb'))
term_dict = dict((v,k) for k,v in id_term_map.items())

In [3]:
file_path = '{}/emotion_documents'.format(data_path)
h5f = h5py.File(file_path + '.h5', 'r')
documents = np.array(h5f['documents'])
seq_len = np.array(h5f['seq_len'])
label = np.array(h5f['label'])
new_idx = np.array(h5f['new_idx'])
nb_words = np.array(h5f['id_term_map_length'])
h5f.close()
label_str = [label_str[idx] for idx in new_idx]

In [4]:
flags = tf.app.flags

flags.DEFINE_integer("nb_words", nb_words, "term number in input sequence(zero mask) [20001]")
flags.DEFINE_integer("maxlen", maxlen, "the max length of input sequence [80]")
flags.DEFINE_integer("embedding_size", 300, "word embedding size [50]")
flags.DEFINE_float("init_scale", 1.0, "init scale for embedding layer")
flags.DEFINE_float("learning_rate", 0.001, "learning rate [0.001]")
#flags.DEFINE_string("opt", 'sgd', "optimizer")
flags.DEFINE_integer("batch_size", 512, "batch size to use during training [128]")
flags.DEFINE_float("clip_gradients", 5.0, "clip gradients to this norm [5.0]")
flags.DEFINE_integer("n_epochs", 50, "number of epoch to use during training [10]")
flags.DEFINE_boolean("epoch_save", True, "save checkpoint or not in each epoch [True]")
flags.DEFINE_integer("print_step", 100, "print step duraing training [100]")
flags.DEFINE_string("logs_dir", "logs/", "logs directory [logs/]")
flags.DEFINE_string("model_dir", "model/", "model directory [model/]")
flags.DEFINE_boolean("dir_clear", False, "clear the log and model directory")
flags.DEFINE_boolean("lr_annealing", False, "use lr annealing or not after each epoch [False]")
flags.DEFINE_integer("gpu_id", 0, "default gpu id [0]")
flags.DEFINE_integer("gpu_num", 4, "gpu_num")

FLAGS = flags.FLAGS

In [ ]:
idxs = np.arange(0, len(seq_len))
label[label<=1] = 0
label[label>=3] = 1
#2387043
idxs = np.append(np.random.choice(np.where(label == 0)[0], 2387043, replace=False),
                 np.random.choice(np.where(label == 1)[0], 2387043, replace=False))
#idxs = np.delete(idxs, np.where(seq_len==1)[0])
#idxs = np.delete(idxs, np.where(label == 2)[0])
#idxs = np.delete(idxs, np.where(label == 3)[0])
#idxs = idxs[0:200000]
train_idxs, test_idxs = train_test_split(idxs, test_size=0.2, random_state=42)
test_idxs = np.sort(test_idxs)

In [ ]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
with tf.Session(graph=graph_to_use, config=config) as session:
    #cnn_model = TextCNN(FLAGS, session, current_task_name='text_cnn_model')
    #cnn_model.build_model(num_classes=len(set(label[idxs])),max_conv_len=7, num_filters=512, dropout_keep_prob=0.5)
    cnn_model = TextMultiCNN(FLAGS, session, current_task_name='text_multi_cnn_model')
    #label[idxs]
    cnn_model.build_model(num_classes = len(set(label[idxs])), num_filters_per_size=256, fully_layers = [1024, 1024])
    cnn_model.build_model_summary()
    display(cnn_model.model_summary())
    cnn_model.run([documents,label], train_idxs, test_idxs)

Initializing


,variable_name,variable_shape,parameters
0,global/Variable:0,[],1
1,gpu/embedding/embedding_layer/embedding_table:0,"[15987, 300]",4796100
2,gpu/conv/conv_1d/he_uniform/W:0,"[7, 300, 256]",537600
3,gpu/conv/conv_1d/B:0,[256],256
4,gpu/conv_1/conv_1d/he_uniform/W:0,"[7, 256, 256]",458752
5,gpu/conv_1/conv_1d/B:0,[256],256
6,gpu/conv_2/conv_1d/he_uniform/W:0,"[3, 256, 256]",196608
7,gpu/conv_2/conv_1d/B:0,[256],256
8,gpu/conv_3/conv_1d/he_uniform/W:0,"[3, 256, 256]",196608
9,gpu/conv_3/conv_1d/B:0,[256],256


Epoch 1 ... training ...
Minibatch 100 / loss: 0.626377
Minibatch 100 / accuracy: 0.654297
Minibatch 200 / loss: 0.557568
Minibatch 200 / accuracy: 0.712891
Minibatch 300 / loss: 0.587483
Minibatch 300 / accuracy: 0.697266


In [7]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
with tf.Session(graph=graph_to_use, config=config) as session:
    cnn_model = TextMultiCNN(FLAGS, session, current_task_name='text_multi_cnn_model')
    #label[idxs]
    cnn_model.build_model(num_classes = len(set(label[idxs])), num_filters_per_size=256, fully_layers = [1024, 1024])
    cnn_model.build_model_summary()
    cnn_model.model_restore()
    display(cnn_model.model_summary())
    _, testMetric, results = cnn_model.model_run([documents,label], 
                                                  test_idxs, run_type='self',
                                                 mode='test', shuffle=False, save_metric=True)

Initializing
INFO:tensorflow:Restoring parameters from model/text_multi_cnn_model.ckpt
Load Model ...


,variable_name,variable_shape,parameters
0,global/Variable:0,[],1
1,gpu/embedding/embedding_layer/embedding_table:0,"[15987, 300]",4796100
2,gpu/conv/conv_1d/he_uniform/W:0,"[7, 300, 256]",537600
3,gpu/conv/conv_1d/B:0,[256],256
4,gpu/conv_1/conv_1d/he_uniform/W:0,"[7, 256, 256]",458752
5,gpu/conv_1/conv_1d/B:0,[256],256
6,gpu/conv_2/conv_1d/he_uniform/W:0,"[3, 256, 256]",196608
7,gpu/conv_2/conv_1d/B:0,[256],256
8,gpu/conv_3/conv_1d/he_uniform/W:0,"[3, 256, 256]",196608
9,gpu/conv_3/conv_1d/B:0,[256],256


Minibatch 100 / loss: 0.592374
Minibatch 100 / accuracy: 0.707031
Minibatch 200 / loss: 0.515164
Minibatch 200 / accuracy: 0.75
Minibatch 300 / loss: 0.613177
Minibatch 300 / accuracy: 0.701172
Minibatch 400 / loss: 0.483553
Minibatch 400 / accuracy: 0.779297
Minibatch 500 / loss: 0.548559
Minibatch 500 / accuracy: 0.734375
Minibatch 600 / loss: 0.422131
Minibatch 600 / accuracy: 0.824219
Minibatch 700 / loss: 0.421453
Minibatch 700 / accuracy: 0.816406
Minibatch 800 / loss: 0.629286
Minibatch 800 / accuracy: 0.705078
Minibatch 900 / loss: 0.48899
Minibatch 900 / accuracy: 0.765625
Minibatch 1000 / loss: 0.167184
Minibatch 1000 / accuracy: 0.986328
Minibatch 1100 / loss: 0.684116
Minibatch 1100 / accuracy: 0.646484
Minibatch 1200 / loss: 0.567101
Minibatch 1200 / accuracy: 0.730469
Minibatch 1300 / loss: 0.252716
Minibatch 1300 / accuracy: 0.908203
Minibatch 1400 / loss: 0.397288
Minibatch 1400 / accuracy: 0.816406
Minibatch 1500 / loss: 0.623627
Minibatch 1500 / accuracy: 0.710938
Min